<a href="https://colab.research.google.com/github/choihoyeon1214/seowoon_coin/blob/main/%EC%BD%94%EC%9D%B8%EA%B0%80%EA%B2%A9%EC%98%88%EC%B8%A1%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !python --version #버전확인

In [ ]:
  !wget https://www.python.org/ftp/python/3.7.1/Python-3.7.1.tgz
  !tar xvfz Python-3.7.1.tgz
  !Python-3.7.1/configure
  !make
  !sudo make install #파이썬 다운그레이드

In [ ]:
 !python --version #버전확인

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import yfinance as yf
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout #필요한 라이브러리 설치

In [ ]:
def StockData(code, start_date):
    try:
        yf.pdr_override()
        df = pdr.get_data_yahoo(f'{code}', start=f'{start_date}')
        df = df[['Open', 'High', 'Low', 'Volume', 'Close']]
        return df
    except Exception as e:
        print(f"Error fetching data for {code}: {e}")
        return None #데이터수집

In [ ]:
stock_code = input("Enter the stock code (ex Apple-AAPL samsung-005930.KS): ").strip()
start_date = input("Enter the start date (2020-01-01): ").strip()

df = StockData(stock_code, start_date)  #데이터수집
if df is not None:
    print(df.head())
else:
    print("Failed to retrieve data.")
    exit()

In [ ]:
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)     #알고리즘 성능개선

In [ ]:
df_x = MinMaxScaler(df)
df_y = df_x[['Close']]

x = df_x.values.tolist()
y = df_y.values.tolist()

In [ ]:
df_x.info()

In [ ]:
df_y.info()

In [ ]:
x[-1]

In [ ]:
y[-1]

In [ ]:
data_x = []
data_y = []
window_size = 10     #데이터 준비
for i in range(len(y) - window_size):
    x2 = x[i : i + window_size]
    y2 = y[i + window_size]
    data_x.append(x2)
    data_y.append(y2)
print("이전 10일 동안 OHLVC : ", x2, "\n다음 날 종가 : ", y2)

In [ ]:
# 훈련용 데이터
train_size = int(len(data_y) * 0.7)
train_x = np.array(data_x[0:train_size])
train_y = np.array(data_y[0:train_size])

# 테스트용 데이터
test_size = len(data_y) - train_size
test_x = np.array(data_x[train_size:len(data_x)])
test_y = np.array(data_y[train_size:len(data_y)])

In [ ]:
model = Sequential()
model.add(LSTM(units=10, activation='relu', return_sequences=True, input_shape=(window_size, 5)))     #모델생성
model.add(Dropout(0.1))
model.add(LSTM(units=10, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=1))
model.summary()

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(train_x, train_y, epochs=60, batch_size=30)
pred_y = model.predict(test_x)

In [ ]:
plt.figure()
#plt.plot(test_y, color='r', label='real stock price')
plt.plot(pred_y, color='b', label='predicted stock price')
#plt.title('Real Stock Price vs Predicted Stock Price')
plt.xlabel('time')
plt.ylabel('stock price')
plt.legend(loc='best')
plt.show() #그래프